In [1]:
import time
from typing import Any
from tqdm import tqdm

import gymnasium as gym
import numpy as np
from gymnasium.envs.registration import register
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

from swarm.swarm_and_agent_env import SwarmAndAgentEnvConfig


In [2]:
register(
    id="SwarmAndAgentEnv",
    entry_point="swarm.swarm_and_agent_env:SwarmAndAgentEnv",
)

In [3]:
def benchmark_env(
    name: str,
    num_resets: int,
    num_steps: int,
    action: Any,
    **kwargs,
) -> np.ndarray:
    """Benchmark environment.

    Return array, where array[r, s+1] = time for step s in after reset r.
    array[r, 0] is the time for the reset.
    """
    env = gym.make(name, **kwargs)

    with tqdm(total=num_resets * (num_steps + 1)) as pbar:
        times = []
        for _ in range(num_resets):
            ts = []

            ts.append(time.perf_counter())
            env.reset()
            ts.append(time.perf_counter())

            pbar.update(1)

            for _ in range(num_steps):
                env.step(action)
                ts.append(time.perf_counter())
                pbar.update(1)

            times.append(ts)

        times = np.diff(np.array(times), axis=1)

        return times


In [52]:
config = SwarmAndAgentEnvConfig()
name = "SwarmAndAgentEnv"

times = benchmark_env(
    name, num_resets=20, num_steps=10000, action=np.zeros(2), config=config
)


100%|██████████| 200020/200020 [05:35<00:00, 595.76it/s]


In [53]:
qs = [0, 0.25, 0.5, 0.75, 1]
values = np.quantile(1 / times[:, 1:], qs, axis=0)

fig = go.Figure(layout_title="FPS")
for q, v in zip(qs, values):
    fig.add_scatter(y=pd.Series(v).rolling(100).mean(), mode="lines", name=q)

fig.add_scatter(
    y=pd.Series(1 / times[:, 1:].mean(axis=0)).rolling(100).mean(),
    mode="lines",
    name="mean",
)
fig
